# This notebook is for FInding the new type of  AuxTel mount errors.

Craig Lage - 28Jan24

In [ ]:
import nest_asyncio
nest_asyncio.apply()
import sys, time, os, asyncio, warnings
from datetime import datetime
import numpy as np
import pandas as pd
import pickle as pkl
import matplotlib.pyplot as plt
%matplotlib inline
from astropy.time import Time, TimeDelta

from lsst_efd_client import EfdClient
from lsst.daf.butler import Butler

import lsst.summit.utils.butlerUtils as butlerUtils
from lsst.summit.utils.utils import dayObsIntToString
from lsst.summit.utils.efdUtils import calcNextDay
from lsst.ts.observing.utilities.decorated_logger import DecoratedLogger
from lsst.rubintv.production import mountTorques
from lsst_efd_client import merge_packed_time_series as mpts

In [ ]:
async def getRotatorAngle(butler, client, dataId):
    expRecord = butlerUtils.getExpRecordFromDataId(butler, dataId)
    tStart = expRecord.timespan.begin.tai.to_value("isot")
    tEnd = expRecord.timespan.end.tai.to_value("isot")
    t_start = Time(tStart, scale="tai")
    t_end = Time(tEnd, scale="tai")
    nasmyth_position = await client.select_time_series("lsst.sal.ATMCS.mount_Nasmyth_Encoders", \
                                            ["*"],  t_start.utc, t_end.utc)
    rot = mpts(nasmyth_position, "nasmyth2CalculatedAngle", stride=1)
    rot_vals = np.array(rot.values[:, 0])
    medrot = np.median(rot_vals)
    return medrot


# Now we run it on all of 2025

In [ ]:
from lsst_efd_client import EfdClient
client = EfdClient('usdf_efd')
butler = butlerUtils.makeDefaultButler("LATISS")
logger = DecoratedLogger.get_decorated_logger()
warnings.filterwarnings('ignore')
startDay = 20250107
Mount_Errors_Classified_Dict = {}
endDay = 20250127
dayObs = startDay
data = {}

while dayObs <= endDay:
    records = butler.registry.queryDimensionRecords("exposure", where="exposure.day_obs=%d"%dayObs)
    print(dayObs, records.count())
    if records.count() < 200:
        dayObs = calcNextDay(dayObs)  # find the next dayObs                                                                                       
        continue
    exposureList = []
    for record in butler.registry.queryDimensionRecords("exposure", where="exposure.day_obs=%d"%dayObs):
        if record.observation_type not in ['bias', 'flat', 'dark'] and record.exposure_time > 2.0:
            exposureList.append(record.id)
    exposureList = sorted(exposureList)
    print(dayObs, len(exposureList))
    for expId in exposureList:
        try:
            seqNum = int(expId - dayObs * 1E5)
            dataId = {'detector':0, 'exposure':expId}
            dict = mountTorques.calculateMountErrors(dataId, butler, client, figure=None, saveFilename=None, logger=logger)
            if dict['az_rms']<0.2 and dict['el_rms']>0.25 and dict['rot_rms']>0.3:
                mData = butler.get('raw.metadata', dataId=dataId)
                rot = await getRotatorAngle(butler, client, dataId)
                az = mData['AZSTART']
                el = mData['ELSTART']
                print(expId, az, el, rot)
                data[expId] = [az, el, rot]
            print(f"{expId} finished successfully")
            sys.stdout.flush()
        except:
            print(f"{expId} failed")
            sys.stdout.flush()
            continue
    print(f"Finished {dayObs}")
    sys.stdout.flush()
    dayObs = calcNextDay(dayObs)  # find the next dayObs                                                                                           
                            

In [ ]:
azs = []
els = []
rots = []
for key in data.keys():
    az = data[key][0]
    if az < 0:
        az += 360.0
    azs.append(az)
    els.append(data[key][1])
    rots.append(data[key][2])
fig, axs = plt.subplots(1,2,figsize=(10,5))
axs[0].set_title("Az/EL")
axs[0].set_xlabel("Azimuth (degrees)")
axs[0].set_ylabel("Elevation (degrees)")
axs[0].scatter(azs, els)
#axs[0].set_xlim(175, 185)
axs[1].set_title("Rotator")
axs[1].set_xlabel("Rotation angle (degrees)")
axs[1].hist(rots)
axs[1].set_xlim(115, 125)
                        
plt.savefig(f"/home/c/cslage/u/AuxTel/mount_graphs/Elevation_Rotator_Errors_28Jan25.png")